In [18]:
import sqlite3 
import pandas as pd
import numpy as np
import requests 
import pysqldf
pd.options.display.float_format = '{:.2f}'.format
# from lib import MongoHandler, WeatherGetter
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [3]:
cur.execute("""SELECT DISTINCT Date, HomeTeam, FTHG, AwayTeam, FTAG, FTR
                FROM teams_in_matches
                JOIN unique_teams
                USING(unique_team_id)
                JOIN matches
                USING (match_id)
                WHERE season IN (SELECT season
                                    FROM matches 
                                    WHERE season == '2011')
            """)

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df = df.sort_values(by=['FTR'])
df.head()

,Date,HomeTeam,FTHG,AwayTeam,FTAG,FTR
0,2012-03-31,Nurnberg,0,Bayern Munich,1,A
673,2011-10-01,Everton,0,Liverpool,2,A
676,2011-10-01,Wolves,1,Newcastle,2,A
677,2011-10-02,Bolton,1,Chelsea,5,A
687,2011-10-15,Wigan,1,Bolton,3,A


In [4]:
unique_dates=cur.execute("SELECT DISTINCT Date FROM Matches WHERE Season IS 2011;").fetchall()

listo=[]
for date in unique_dates:
    for datetime in date:
        listo.append(datetime + "T13:00:00")
listo

weather_dates={}
for date in listo:
    result = requests.get(f"https://api.darksky.net/forecast/958870d7f05d6ce07081ed0698de9eca/52.520008,13.404954,{date}").json()
#     weather_type=call_weather['currently']['summary']
    weather_dates[date]=result['currently'].get('icon') if result['currently'].get('icon') else result['currently'].get('summary')




weather_dates

weather_dates_day={}
for date in weather_dates:
   weather_dates_day[date[:10]]=weather_dates[date]
weather_dates_day

{'2012-03-31': 'partly-cloudy-day',
 '2011-12-11': 'partly-cloudy-day',
 '2011-08-13': 'partly-cloudy-day',
 '2011-11-27': 'partly-cloudy-day',
 '2012-02-18': 'partly-cloudy-day',
 '2012-01-20': 'partly-cloudy-day',
 '2012-02-04': 'partly-cloudy-day',
 '2012-04-21': 'partly-cloudy-day',
 '2011-09-18': 'partly-cloudy-day',
 '2011-10-23': 'Clear',
 '2011-10-01': 'Clear',
 '2012-03-03': 'Clear',
 '2011-08-27': 'partly-cloudy-day',
 '2012-03-17': 'Clear',
 '2011-11-06': 'Clear',
 '2012-05-05': 'partly-cloudy-day',
 '2012-04-11': 'partly-cloudy-day',
 '2011-12-17': 'partly-cloudy-day',
 '2012-02-03': 'partly-cloudy-day',
 '2011-10-29': 'Clear',
 '2012-01-22': 'partly-cloudy-day',
 '2011-12-03': 'cloudy',
 '2012-04-14': 'partly-cloudy-day',
 '2012-03-25': 'Clear',
 '2012-03-10': 'partly-cloudy-day',
 '2012-04-07': 'partly-cloudy-day',
 '2011-11-19': 'Clear',
 '2011-10-14': 'partly-cloudy-day',
 '2011-09-24': 'clear-day',
 '2012-04-28': 'Clear',
 '2011-12-18': 'partly-cloudy-day',
 '2012-03-0

In [5]:
df['weather'] = df['Date'].map(weather_dates_day)
df.head()

,Date,HomeTeam,FTHG,AwayTeam,FTAG,FTR,weather
0,2012-03-31,Nurnberg,0,Bayern Munich,1,A,partly-cloudy-day
673,2011-10-01,Everton,0,Liverpool,2,A,Clear
676,2011-10-01,Wolves,1,Newcastle,2,A,Clear
677,2011-10-02,Bolton,1,Chelsea,5,A,Clear
687,2011-10-15,Wigan,1,Bolton,3,A,clear-day


In [6]:
df['rain_home_win'] =np.where(np.logical_and(df['FTR']=='H',df['weather']=='rain'),1,0)

In [7]:
df['rain_away_win'] =np.where(np.logical_and(df['FTR']=='A',df['weather']=='rain'),1,0)

In [20]:
rain_wins = []
for x in df.index:
    win = df['rain_home_win'][x] + df['rain_away_win'][x]
    rain_wins.append(win)

df['rain_wins'] = rain_wins

In [9]:
#HOME WINS
cur.execute("""SELECT DISTINCT match_id, HomeTeam 
                FROM teams_in_matches
                JOIN unique_teams
                USING(unique_team_id)
                JOIN matches
                USING (match_id)
                WHERE season IN (SELECT season
                                    FROM matches 
                                    WHERE season == '2011')
               AND
               ftr = 'H'   
            """)

home_df = pd.DataFrame(cur.fetchall())
home_df.columns = [i[0] for i in cur.description]
home_df = home_df.groupby(home_df['HomeTeam']).count()

In [10]:
df_wins = home_df.join(away_df, lsuffix='_Home', rsuffix='_Away')
df_wins.columns = ['Home_Wins', 'Away_Wins']

In [11]:
#TOTAL WINS
total_wins = []
home_wins = df_wins['Home_Wins']
away_wins = df_wins['Away_Wins']


for item in df_wins.index:
    wins = home_wins[item] + away_wins[item]
    total_wins.append(wins)

df_wins['Total_Wins'] = total_wins

In [12]:
#DRAWS
cur.execute("""SELECT DISTINCT match_id, HomeTeam, FTHG, AwayTeam, FTAG
                FROM teams_in_matches
                JOIN unique_teams
                USING(unique_team_id)
                JOIN matches
                USING (match_id)
                WHERE season IN (SELECT season
                                    FROM matches 
                                    WHERE season == '2011')
               AND
               ftr = 'D'   
            """)

draws_df = pd.DataFrame(cur.fetchall())
draws_df.columns = [i[0] for i in cur.description]
draws_df = draws_df.groupby(draws_df['AwayTeam'])
draws_df.head()

# home = draws_df['HomeTeam'].count()
# home
away = draws_df['AwayTeam'].count()
df_wins['Total_Draws'] = away

In [13]:
#Home Losses
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q ="""SELECT COUNT(HomeTeam), HomeTeam
        FROM df
        WHERE FTR = 'A'
        GROUP BY HomeTeam       
    """

results = pysqldf(q)
df3 = pd.DataFrame(results)

In [14]:
#Away Losses
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q ="""SELECT COUNT(AwayTeam)
        FROM df
        WHERE FTR = 'H'
        GROUP BY AwayTeam        
    """

results = pysqldf(q)
df4 = pd.DataFrame(results)

In [15]:
#Home Wins
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q ="""SELECT COUNT(HomeTeam)
        FROM df
        WHERE FTR = 'H'
        GROUP BY HomeTeam       
    """

results = pysqldf(q)
df5 = pd.DataFrame(results)

In [16]:
#Away Wins
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q ="""SELECT COUNT(AwayTeam)
        FROM df
        WHERE FTR = 'A'
        GROUP BY AwayTeam       
    """

results = pysqldf(q)
df6 = pd.DataFrame(results)

In [17]:
df_losses = df3.join(df4)
df_losses = df_losses.join(df5, rsuffix='HomeWins' )
df_losses = df_losses.join(df6, rsuffix='AwayWins')
df_losses

,COUNT(HomeTeam),HomeTeam,COUNT(AwayTeam),COUNT(HomeTeam)HomeWins,COUNT(AwayTeam)AwayWins
0,7,Aachen,8,4,2
1,3,Arsenal,7,12,9
2,8,Aston Villa,6,4,3
3,4,Augsburg,8,6,2
4,2,Bayern Munich,5,14,9
5,12,Blackburn,11,6,2
6,7,Bochum,10,7,3
7,11,Bolton,11,4,6
8,3,Braunschweig,6,6,4
9,4,Chelsea,6,12,6


In [21]:
Team = []
Wins = []
Losses = []

for x in df_losses.index:
    Team.append(df_losses['HomeTeam'][x])
    Wins.append(df_losses['COUNT(HomeTeam)HomeWins'][x] + df_losses['COUNT(AwayTeam)AwayWins'][x])
    Losses.append(df_losses['COUNT(HomeTeam)'][x] + df_losses['COUNT(AwayTeam)'][x])

   

for x in Team:
    dict = {}
    dict[x] = "Team:"
#     print("Team: ", Team[x])
#     print("Wins: ", Wins[x])
#     print("Losses", Losses[x])
#     print("")

dict

NameError: name 'df_losses' is not defined

In [23]:
df_losses = home_df.join(away_df, lsuffix='_Home', rsuffix='_Away')
df_losses.columns = ['Home_Losses', 'Away_Losses']

NameError: name 'home_df' is not defined

In [24]:
#TOTAL LOSSES
total_losses = []
home_losses = df_losses['Home_Losses']
away_losses = df_losses['Away_Losses']


for item in df_losses.index:
    loss = home_losses[item] + away_losses[item]
    total_losses.append(loss)

df_losses['Total_losses'] = total_losses
df_wins['Total_Losses'] = df_losses['Total_losses']

NameError: name 'df_losses' is not defined

In [22]:
df_wins.head()

,Home_Wins,Away_Wins,Total_Wins,Total_Draws,Total_Losses
HomeTeam,,,,,
Aachen,4,2,6,7,6
Arsenal,12,9,21,3,21
Aston Villa,4,3,7,10,7
Augsburg,6,2,8,7,8
Bayern Munich,14,9,23,3,23


In [23]:
for x in df_wins.index:
    print("['Team' : '{}']".format(x))

['Team' : 'Aachen']
['Team' : 'Arsenal']
['Team' : 'Aston Villa']
['Team' : 'Augsburg']
['Team' : 'Bayern Munich']
['Team' : 'Blackburn']
['Team' : 'Bochum']
['Team' : 'Bolton']
['Team' : 'Braunschweig']
['Team' : 'Chelsea']
['Team' : 'Cottbus']
['Team' : 'Dortmund']
['Team' : 'Dresden']
['Team' : 'Duisburg']
['Team' : 'Ein Frankfurt']
['Team' : 'Erzgebirge Aue']
['Team' : 'Everton']
['Team' : 'FC Koln']
['Team' : 'Fortuna Dusseldorf']
['Team' : 'Frankfurt FSV']
['Team' : 'Freiburg']
['Team' : 'Fulham']
['Team' : 'Greuther Furth']
['Team' : 'Hamburg']
['Team' : 'Hannover']
['Team' : 'Hansa Rostock']
['Team' : 'Hertha']
['Team' : 'Hoffenheim']
['Team' : 'Ingolstadt']
['Team' : 'Kaiserslautern']
['Team' : 'Karlsruhe']
['Team' : 'Leverkusen']
['Team' : 'Liverpool']
['Team' : 'M'gladbach']
['Team' : 'Mainz']
['Team' : 'Man City']
['Team' : 'Man United']
['Team' : 'Munich 1860']
['Team' : 'Newcastle']
['Team' : 'Norwich']
['Team' : 'Nurnberg']
['Team' : 'Paderborn']
['Team' : 'QPR']
['Team'